In [ ]:
import pandas as pd
import re
import json

import openai
from openai import OpenAI
import google.generativeai as genai
from google.colab import userdata

# Dataset

In [ ]:
df = pd.read_csv('dataset.csv')

In [ ]:
df

,problem,solution,message,status_is_correct,source,name
0,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle
1,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Runtime error,False,URI Online Judge,Area of a Circle
2,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle
3,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle
4,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle
5,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Runtime error,False,URI Online Judge,Area of a Circle
6,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Accepted,True,URI Online Judge,Area of a Circle
7,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle
8,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle
9,Professor Rolien and their Computer Science st...,def camisetas():\n while True:\n N =...,Presentation error,False,URI Online Judge,T-Shirts


# Generate Prompts

In [ ]:
example_msg = """
Example output:
    {
      "is_correct": false,
      "hints": [
        {
          "line_number": 3,
          "code_line": "x = x + 2",
          "hint": "The variable 'x' is used before being initialized."
        },
        {
          "line_number": 5,
          "code_line": "if x > 0:",
          "hint": "The loop condition does not account for edge cases."
        }
      ]
    }

If the code is correct, return:
{
  "is_correct": true,
  "hints": []
}
"""

In [ ]:
def generate_prompts_from_dataframe(df):
    """
    Generate a list of prompts for each row in the dataframe.

    Parameters:
        df (pd.DataFrame): A dataframe containing 'problem' and 'solution' columns.

    Returns:
        list: A list of prompts with placeholders replaced by dataframe values.
    """
    prompt_template = """
    I want you to act as a programming teacher that helps students to solve a programming assignment in Python 3.11.
    Review the student's code for the following problem:

    Problem: {problem}

    Student Code: {solution}

    Identify if the code executes correctly and fulfills the problem's requirements. If not, provide a JSON object with:
    - "is_correct": A boolean indicating whether the code is correct.
    - "hints": An array of objects, each with:
      - "line_number": The line number of the issue.
      - "code_line": The code line with the issue.
      - "hint": A short explanation of what is wrong.
    """

    prompts = []
    for _, row in df.iterrows():
        prompt = prompt_template.format(problem=row['problem'], solution=row['solution'])
        prompt += example_msg
        prompts.append(prompt.strip())  # Strip to remove leading/trailing whitespace
    return prompts


In [ ]:
prompts = generate_prompts_from_dataframe(df)

In [ ]:
print(prompts[44])  # Print the first prompt

I want you to act as a programming teacher that helps students to solve a programming assignment in Python 3.11.
    Review the student's code for the following problem:

    Problem: In chats, it is very common among young people and adolescents use strings of letters, which often seem random to represent laughter. Some common examples are:

huaauhahhuahau
hehehehe
ahahahaha
jaisjjkasjksjjskjakijs
huehuehue

Claudia is a young programmer who was intrigued by the sound of "digital laughter." Some of them she can not even pronounce! But she realized that some of them seem better convey the feeling of laughter than others. The first thing she noticed is that the consonants do not interfere in how digital laughter influence the feeling of transmission. The second thing she noticed is that the funniest digital laughter are those in which the sequences of vowels are the same when read in the natural order (from left to right) or in reverse order (from right to left), ignoring consonants. Fo

In [ ]:
def extract_feedback(df_with_feedback):
    """
    Extracts feedback JSON from a column and creates a new DataFrame with structured feedback information.

    Parameters:
        df_with_feedback (pd.DataFrame): A DataFrame with the column 'feedback' containing JSON inside text.

    Returns:
        pd.DataFrame: A new DataFrame with structured feedback data.
    """
    records = []

    # Add an "id" column to the original DataFrame
    df_with_feedback = df_with_feedback.reset_index().rename(columns={"index": "id"})

    for _, row in df_with_feedback.iterrows():
        feedback_text = row['feedback']
        id_feedback = row['id']

        # Extract JSON content within the triple backticks pattern
        match = re.search(r"```json\n(.*?)\n```", feedback_text, re.DOTALL)
        if match:
            try:
                feedback_json = json.loads(match.group(1))
                is_correct = feedback_json.get("is_correct", False)
                hints = feedback_json.get("hints", [])

                # Create a record for each hint
                if len(hints)==0:
                    records.append({
                        "id_feedback": id_feedback,
                        "is_correct": is_correct,
                        "line_number": None,
                        "code_line": None,
                        "hint": None
                    })
                for hint in hints:
                    records.append({
                        "id_feedback": id_feedback,
                        "is_correct": is_correct,
                        "line_number": hint.get("line_number"),
                        "code_line": hint.get("code_line"),
                        "hint": hint.get("hint")
                    })
            except json.JSONDecodeError:
                print(f"Invalid JSON found in feedback for id {id_feedback}")
                records.append({
                    "id_feedback": id_feedback,
                    "is_correct": None,
                    "line_number": None,
                    "code_line": None,
                    "hint": "Invalid JSON in feedback"
                })
        elif feedback_text.startswith("{") and feedback_text.endswith("}"):
            try:
                feedback_json = json.loads(feedback_text)
                is_correct = feedback_json.get("is_correct", False)
                hints = feedback_json.get("hints", [])
                # Create a record for each hint
                if len(hints)==0:
                    records.append({
                        "id_feedback": id_feedback,
                        "is_correct": is_correct,
                        "line_number": None,
                        "code_line": None,
                        "hint": None
                    })
                for hint in hints:
                    records.append({
                        "id_feedback": id_feedback,
                        "is_correct": is_correct,
                        "line_number": hint.get("line_number"),
                        "code_line": hint.get("code_line"),
                        "hint": hint.get("hint")
                    })
            except json.JSONDecodeError:
                print(f"Invalid JSON found in feedback for id {id_feedback}")
                records.append({
                    "id_feedback": id_feedback,
                    "is_correct": None,
                    "line_number": None,
                    "code_line": None,
                    "hint": "Invalid JSON in feedback"
                })
        else:
            print(f"No JSON found in feedback for id {id_feedback}")
            records.append({
                "id_feedback": id_feedback,
                "is_correct": None,
                "line_number": None,
                "code_line": None,
                "hint": "No JSON found in feedback"
            })

    # Create a new DataFrame from the records
    feedback_df = pd.DataFrame(records, columns=["id_feedback", "is_correct", "line_number", "code_line", "hint"])
    return feedback_df


# OpenAI Models

In [ ]:
def get_feedback_from_openai(df, api_key, model):
    """
    Sends prompts to OpenAI's API and stores the responses in a new dataframe column.

    Parameters:
        df (pd.DataFrame): A dataframe containing 'problem' and 'solution' columns.
        api_key (str): OpenAI API key.
        model (str): OpenAI model to use for generating responses.

    Returns:
        pd.DataFrame: DataFrame with a new column 'feedback' containing the JSON responses.
    """
    # Set OpenAI API key
    client = OpenAI(api_key = api_key)
    # client = OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

    # Generate prompts
    prompts = generate_prompts_from_dataframe(df)

    # Initialize feedback column
    feedback_list = []

    for i, prompt in enumerate(prompts):
        try:
            # Call OpenAI's API
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                max_tokens=2048,
            )
            # Parse the response content
            feedback = response.choices[0].message.content.strip()
            feedback_list.append(feedback)
        except Exception as e:
            # Handle errors by appending the exception message
            feedback_list.append(f"Error: {e}")
            print(f"Error processing row {i}: {e}")

    # Add feedback column to the DataFrame
    df['feedback'] = feedback_list

    return df

In [ ]:
api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Replace with your actual OpenAI API key
# df_with_feedback = get_feedback_from_openai(df, api_key, model="gpt-4o-mini")
# df_with_feedback = get_feedback_from_openai(df, api_key, model="gpt-4o")
df_with_feedback = get_feedback_from_openai(df, api_key, model="gpt-4-turbo")

In [ ]:
df_with_feedback

,problem,solution,message,status_is_correct,source,name,feedback,model_choice
0,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle,The student's code provided for calculating th...,False
1,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Runtime error,False,URI Online Judge,Area of a Circle,The student's code provided has a few issues t...,False
2,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle,"```json\n{\n ""is_correct"": false,\n ""hints"":...",False
3,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle,The student's code provided for calculating th...,False
4,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle,"```json\n{\n ""is_correct"": false,\n ""hints"":...",False
5,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Runtime error,False,URI Online Judge,Area of a Circle,The student's code provided for calculating th...,False
6,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Accepted,True,URI Online Judge,Area of a Circle,The student's code provided for calculating th...,True
7,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle,"```json\n{\n ""is_correct"": false,\n ""hints"":...",False
8,The formula to calculate the area of a circumf...,raio = float(input())\n \n pi = 3.14159\n \n a...,Wrong answer,False,URI Online Judge,Area of a Circle,The student's code has a minor issue with the ...,False
9,Professor Rolien and their Computer Science st...,def camisetas():\n while True:\n N =...,Presentation error,False,URI Online Judge,T-Shirts,The student's code attempts to solve the probl...,False


In [ ]:
df_with_feedback.to_csv("dataset_with_feedback.csv", index=False)

In [ ]:
print(df_with_feedback['feedback'].values[6])

The student's code provided for calculating the area of a circle based on the radius input and the formula \( A = \pi \times R^2 \) is as follows:

```python
raio = float(input())

pi = 3.14159

area = pi * (raio ** 2)

print('A={0:0.4f}'.format(area))
```

Upon reviewing the code, it appears that the student has correctly implemented the formula to calculate the area of the circle. The code correctly reads a floating-point number as input, calculates the area using the given value of \(\pi\), and prints the result formatted to four decimal places. The output also includes the required prefix "A=" and the result is formatted as specified.

The code also correctly handles the output by using Python's string formatting to ensure the area is displayed with four decimal places. The use of `print('A={0:0.4f}'.format(area))` is appropriate for the required output format.

Therefore, the code executes correctly and fulfills the problem's requirements. Here is the JSON object indicating the co

In [ ]:
print(df_with_feedback['feedback'].values[1])

The student's code provided has a few issues that need to be addressed to meet the problem's requirements and ensure proper execution. Here's the analysis:

1. The code is missing a closing parenthesis on the print statement.
2. The output format does not match the specified requirement of four decimal places.
3. The code does not ensure a newline character at the end of the output.

Here's the JSON object detailing these issues:

```json
{
  "is_correct": false,
  "hints": [
    {
      "line_number": 5,
      "code_line": "print('A=' + str(area)",
      "hint": "Missing a closing parenthesis and does not format the output to four decimal places."
    },
    {
      "line_number": 5,
      "code_line": "print('A=' + str(area)",
      "hint": "Ensure the output has exactly four decimal places and includes a newline character."
    }
  ]
}
```

To correct the code, the student should modify the print statement to ensure proper formatting and include the newline character implicitly hand

In [ ]:
processed_df = extract_feedback(df_with_feedback)

In [ ]:
processed_df

,id_feedback,is_correct,line_number,code_line,hint
0,0,False,6.0,"print('A=',area)",The output should be formatted to four decimal...
1,1,False,5.0,print('A=' + str(area),Missing a closing parenthesis and does not for...
2,1,False,5.0,print('A=' + str(area),Ensure the output has exactly four decimal pla...
3,2,False,6.0,print('A=' + str(area)),The output should have four decimal places. Us...
4,2,False,6.0,print('A=' + str(area)),Ensure to include a newline character at the e...
...,...,...,...,...,...
59,43,False,3.0,if len(entrada) > maximo:,This length check is unnecessary as the proble...
60,43,False,6.0,entrada = entrada.lower(),This line is redundant since the input is guar...
61,44,False,9.0,if any(vogal in entrada for vogal in 'aeiou'):...,This line incorrectly checks if any vowels are...
62,44,False,9.0,else: print('N'),"This line prints 'N' if no vowels are present,..."


In [ ]:
processed_df.to_csv("dataset_processed_feedback.csv", index=False)

In [ ]:
# Remove duplicates based on 'id_feedback' column, keeping the first occurrence
df_model_choice = processed_df.drop_duplicates(subset=['id_feedback'], keep='first')
df_model_choice = df_model_choice.reset_index()
df_model_choice

,index,id_feedback,is_correct,line_number,code_line,hint
0,0,0,False,6.0,"print('A=',area)",The output should be formatted to four decimal...
1,1,1,False,5.0,print('A=' + str(area),Missing a closing parenthesis and does not for...
2,3,2,False,6.0,print('A=' + str(area)),The output should have four decimal places. Us...
3,5,3,False,6.0,"print('A=' + str(round(area,4)))",The output may not always display four decimal...
4,6,4,False,5.0,"print('A=' + str(round(area,5)))",The output should have exactly four decimal pl...
5,7,5,True,NaN,None,None
6,8,6,True,NaN,None,None
7,9,7,False,5.0,"print('A=',area)",The output format does not match the required ...
8,10,8,False,6.0,"print('A=""{:.4f}'.format(area))",Remove the quotation marks around the formatte...
9,11,9,False,14.0,"camisas.sort(key=lambda x: (x['cor'], -ord(x['...",Sorting by size using ASCII values is incorrec...


In [ ]:
df_with_feedback['model_choice'] = df_model_choice['is_correct']

In [ ]:
df_with_feedback.to_csv("dataset_with_feedback.csv", index=False)

In [ ]:
df_with_feedback.value_counts('status_is_correct')

,count
status_is_correct,
False,26
True,19


In [ ]:
df_with_feedback.value_counts('model_choice')

,count
model_choice,
False,29
True,16


# Google Model

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import time

def get_feedback_from_google(df, api_key, model_name='gemini-1.5-pro'):
    """
    Sends prompts to Google's API and stores the responses in a new dataframe column.

    Parameters:
        df (pd.DataFrame): A dataframe containing 'problem' and 'solution' columns.
        api_key (str): Google API key.
        model (str): Google model to use for generating responses.

    Returns:
        pd.DataFrame: DataFrame with a new column 'feedback' containing the JSON responses.
    """
    # Set Google API key
    # GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    # genai.configure(api_key=GOOGLE_API_KEY)
    genai.configure(api_key=api_key)

    # Generate prompts
    prompts = generate_prompts_from_dataframe(df)

    # Initialize feedback column
    feedback_list = []

    # #Initialize the Generative Model
    # model = genai.GenerativeModel(model_name)
    # # Generation Config
    # config = genai.GenerationConfig(
    #     max_output_tokens=2048, temperature=0
    # )


    for i, prompt in enumerate(prompts):
        print(i)
        if ((4+1) % 5)==0:
            print("Waiting for 2 minutes before processing the next batch...")
            time.sleep(120)  # Wait for 120 seconds (2 minutes)
        #Initialize the Generative Model
        model = genai.GenerativeModel(model_name)
        # Generation Config
        config = genai.GenerationConfig(
            max_output_tokens=2048, temperature=0
        )
        try:
            # Call Google's API
            response = model.generate_content(prompt, generation_config=config)
            # Parse the response content
            feedback = response.text.strip()
            feedback_list.append(feedback)
        except Exception as e:
            # Handle errors by appending the exception message
            feedback_list.append(f"Error: {e}")
            print(f"Error processing row {i}: {e}")

    # Add feedback column to the DataFrame
    df['feedback'] = feedback_list

    return df

In [ ]:
api_key = "xxxxxxxxxxxxxx"  # Replace with your actual Google API key
df_with_feedback = get_feedback_from_google(df, api_key, model_name="gemini-1.5-pro")